In [ ]:
!pip install l5kit

  Using cached l5kit-1.5.0-py3-none-any.whl (156 kB)
  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for opencv-contrib-python-headless from https://files.pythonhosted.org/packages/c6/61/3b6585f22d59fbf11d9bff0d96a6570a8862c00bc0bbb76efc13bc8cef4c/opencv_contrib_python_headless-4.8.0.74-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pymap3d-3.0.1-py3-none-any.whl (55 kB)
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Using cached transforms3d-0.4.1.tar.gz (1.4 MB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for zarr from https://files.pythonhosted.org/packages/57/8e/b74c2a80c7df474c300bab3b761c35d6cfa6468d6d23f891d518ec7c828e/zarr-2.16.0-py3-none-any.whl.metadata
  Using cached zarr-2.16.0-py3-none-any.whl.metadata (5.4 kB)
  Using c

In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   27G   81G  25% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.1G  887M  55% /usr/sbin/docker-init
tmpfs           6.4G   16M  6.4G   1% /var/colab
/dev/sda1        70G   55G   16G  78% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [83]:
!git clone https://github.com/woven-planet/l5kit.git
%cd l5kit/l5kit


fatal: destination path 'l5kit' already exists and is not an empty directory.
[Errno 2] No such file or directory: 'l5kit/l5kit'
/content/l5kit/l5kit


In [84]:
!pipenv sync --dev

/bin/bash: line 1: pipenv: command not found


In [85]:
!pip install --upgrade pip

In [86]:
!pip install -e ."[dev]"

Obtaining file:///content/l5kit/l5kit
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached opencv_contrib_python_headless-4.7.0.72-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (55.3 MB)
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Using cached Shapely-1.8.5.post1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.0 MB)
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Using cached gym-0.22.0.tar.gz (631 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached typed_ast-1.5.4-cp310-

In [87]:
!pip install zarr

In [88]:
# l5kit dependencies
!pip install pymap3d==2.1.0
!pip install protobuf==3.12.2

!pip install ptable

!pip uninstall -y typing

# The modified l5kit from my github repo (gpu branch)
!pip install --no-dependencies git+https://github.com/pestipeti/lyft-l5kit.git@gpu#subdirectory=l5kit

  Cloning https://github.com/pestipeti/lyft-l5kit.git (to revision gpu) to /tmp/pip-req-build-60v04ro2
  Running command git clone --filter=blob:none --quiet https://github.com/pestipeti/lyft-l5kit.git /tmp/pip-req-build-60v04ro2
  Running command git checkout -b gpu --track origin/gpu
  Switched to a new branch 'gpu'
  Branch 'gpu' set up to track remote branch 'gpu' from 'origin'.
  Resolved https://github.com/pestipeti/lyft-l5kit.git to commit 7fc21b04839289794327cb27f21067391e16723e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [89]:
!pip install pymap3d

In [90]:
!pip install transforms3d

In [108]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as mpt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm

import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import matplotlib.pyplot as mpt

import os
import random
import time

import warnings
warnings.filterwarnings("ignore")

In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
l5kit.__version__

'1.1.0'

In [111]:
def setting_seed_value(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

setting_seed_value(42)

## Configs

In [112]:
# --- Lyft configs ---
configuration = {
    'format_version': 4,
    'data_path': "/content/drive/MyDrive/lyft-motion-prediction-autonomous-vehicles",
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.5,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.5,
        'model_name': "model_resnet34_output",
        'lr': 1e-3,
        'weight_path': "/content/drive/MyDrive/lyft_pretrained_model/model_multi_update_lyft_public.pth",
        'train': True,
        'predict': True,
        "render_ego_history": True,
        "step_time": 0.1
    },

    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': '/content/drive/MyDrive/lyft-motion-prediction-autonomous-vehicles/aerial_map/aerial_map.png',
        'semantic_map_key': '/content/drive/MyDrive/lyft-motion-prediction-autonomous-vehicles/semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
        "set_origin_to_bottom": True,
        "disable_traffic_light_faces": True
    },

    'Training_data_loader': {
        'key': '/content/drive/MyDrive/lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },

    'Testing_data_loader': {
        'key': '/content/drive/MyDrive/lyft-motion-prediction-autonomous-vehicles/scenes/test.zarr',
        'batch_size': 32,
        'shuffle': True,
        'num_workers': 4
    },

    'Training_params': {
        'max_num_steps': 101,
        'checkpoint_every_n_steps': 20,
    }
}

## Load the train and test data

In [113]:
# set env variable for data
Input_Direct = configuration["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = Input_Direct
datamanager = LocalDataManager(None)

In [114]:
# ===== INIT TRAIN DATASET============================================================
Training_configuration = configuration["Training_data_loader"]
rasterizer_bulding = build_rasterizer(configuration, datamanager)
Training_zarr = ChunkedDataset(datamanager.require(Training_configuration["key"])).open()
Training_dataset = AgentDataset(configuration, Training_zarr, rasterizer_bulding)
Training_dataloader = DataLoader(Training_dataset, shuffle=Training_configuration["shuffle"], batch_size=Training_configuration["batch_size"],
                             num_workers=Training_configuration["num_workers"])
print("==================================TRAIN DATA==================================")
print(Training_dataset)

==================================TRAIN DATA==================================
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [115]:
#====== INIT TEST DATASET=============================================================
Testing_configuration = configuration["Testing_data_loader"]
rasterizer_bulding = build_rasterizer(configuration, datamanager)
Testing_zarr = ChunkedDataset(datamanager.require(Testing_configuration["key"])).open()
Testing_mask = np.load(f"{Input_Direct}/scenes/mask.npz")["arr_0"]
Testing_dataset = AgentDataset(configuration, Testing_zarr, rasterizer_bulding, agents_mask=Testing_mask)
Testing_dataloader = DataLoader(Testing_dataset,shuffle=Testing_configuration["shuffle"],batch_size=Testing_configuration["batch_size"],
                             num_workers=Testing_configuration["num_workers"])
print("==================================TEST DATA==================================")
print(Testing_dataset)

==================================TEST DATA==================================
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   11314    |  1131400   |  88594921  |    7854144    |      31.43      |        100.00        |        78.31         |        10.00         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


## Simple visualization

Let us visualize how an input to the model looks like.

In [116]:
def visualize_trajectory(dataset, index, title="target_positions movement with draw_trajectory"):
    data = dataset[index]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    draw_trajectory(im, target_positions_pixels, TARGET_POINTS_COLOR, radius=1, yaws=data["target_yaws"])

    mpt.title(title)
    mpt.imshow(im[::-1])
    mpt.show()

In [118]:
mpt.figure(figsize = (8,6))
visualize_trajectory(Training_dataset, index=90)

## Loss function

In [119]:

import numpy as np

import torch
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

## Model

Next we define the baseline model. Note that this model will return three possible trajectories together with confidence score for each trajectory.

In [120]:
class LyftMultiModel(nn.Module):

    def __init__(self, configuration: Dict, num_modes=3):
        super().__init__()

        architecture = configuration["model_params"]["model_architecture"]
        backbone = eval(architecture)(pretrained=True, progress=True)
        self.backbone = backbone

        num_history_channels = (configuration["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )

        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets

        if architecture == "resnet50":
            backbone_out_features = 2048
        else:
            backbone_out_features = 512

        # X, Y coords for the future positions (output shape: batch_sizex50x2)
        self.future_len = configuration["model_params"]["future_num_frames"]
        num_targets = 2 * self.future_len

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.num_preds = num_targets * num_modes
        self.num_modes = num_modes

        self.logit = nn.Linear(4096, out_features=self.num_preds + num_modes)

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.head(x)
        x = self.logit(x)

        # pred (batch_size)x(modes)x(time)x(2D coords)
        # confidences (batch_size)x(modes)
        bs, _ = x.shape
        pred, confidences = torch.split(x, self.num_preds, dim=1)
        pred = pred.view(bs, self.num_modes, self.future_len, 2)
        assert confidences.shape == (bs, self.num_modes)
        confidences = torch.softmax(confidences, dim=1)
        return pred, confidences

In [121]:
def forward(data, model, device, criterion = pytorch_neg_multi_log_likelihood_batch):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    preds, confidences = model(inputs)
    loss = criterion(targets, preds, confidences, target_availabilities)
    return loss, preds, confidences

Now let us initialize the model and load the pretrained weights. Note that since the pretrained model was trained on GPU, you also need to enable GPU when running this notebook.

In [122]:
# ==== INIT MODEL=================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LyftMultiModel(configuration)

weight_path = configuration["model_params"]["weight_path"]
if weight_path:
    checkpoint = torch.load(weight_path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint)

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=configuration["model_params"]["lr"])
print(f'device {device}')

device cpu


In [123]:
print(model)

LyftMultiModel(
  (backbone): ResNet(
    (conv1): Conv2d(25, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

## Training loop

Next let us implement the training loop, when the **train** parameter is set to True.

In [124]:
# ==== TRAINING LOOP =========================================================
if configuration["model_params"]["train"]:

    tr_it = iter(Training_dataloader)
    progress_bar = tqdm(range(configuration["Training_params"]["max_num_steps"]))
    num_iter = configuration["Training_params"]["max_num_steps"]
    losses_train = []
    iterations = []
    metrics = []
    times = []
    model_name = configuration["model_params"]["model_name"]
    start = time.time()
    for i in progress_bar:
        try:
            data = next(tr_it)
        except StopIteration:
            tr_it = iter(Training_dataloader)
            data = next(tr_it)
        model.train()
        torch.set_grad_enabled(True)

        loss, _, _ = forward(data, model, device)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses_train.append(loss.item())

        progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")
        if i % configuration['Training_params']['checkpoint_every_n_steps'] == 0:
            torch.save(model.state_dict(), f'{model_name}_{i}.pth')
            iterations.append(i)
            metrics.append(np.mean(losses_train))
            times.append((time.time()-start)/60)

    results = pd.DataFrame({'iterations': iterations, 'metrics (avg)': metrics, 'elapsed_time (mins)': times})
    results.to_csv(f"Training_metrics_{model_name}_{num_iter}.csv", index = False)
    print(f"Total training time is {(time.time()-start)/60} mins")
    print(results.head())

## Prediction

In [107]:
# ==== EVAL LOOP ================================================================
if configuration["model_params"]["predict"]:

    model.eval()
    torch.set_grad_enabled(True)

    # store information for evaluation
    future_coords_offsets_pd = []
    timestamps = []
    confidences_list = []
    agent_ids = []

    progress_bar = tqdm(Testing_dataloader)
    print(progress_bar)

    for data in progress_bar:

        _, preds, confidences = forward(data, model, device)

        #fix for the new environment
        preds = preds.detach().cpu().numpy()
        world_from_agents = data["world_from_agent"].numpy()
        centroids = data["centroid"].numpy()
        coords_offset = []

        # convert into world coordinates and compute offsets
        for idx in range(len(preds)):
            for mode in range(3):
                preds[idx, mode, :, :] = transform_points(preds[idx, mode, :, :], world_from_agents[idx]) - centroids[idx][:2]

        future_coords_offsets_pd.append(preds.copy())
        # Convert confidences tensor to a NumPy array without gradients
        confidences_np = confidences.detach().cpu().numpy()
        confidences_list.append(confidences_np.copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy())

In [ ]:

pred_path = 'prediction1.csv'
write_pred_csv(pred_path,
           timestamps=np.concatenate(timestamps),
           track_ids=np.concatenate(agent_ids),
           coords=np.concatenate(future_coords_offsets_pd),
           confs = np.concatenate(confidences_list)
          )

In [ ]:
import pandas as pd, numpy as np

In [ ]:
df = pd.read_csv("./prediction1.csv")

In [ ]:
df_issa1 = df.copy()